## 2.2 - Model Owner chooses a model, creates an activity log

They use a model and ask it to autocomplete gender based prompts from an AlexaAI dataset. 

The model used in this case was GPT2, but we can replace it with [something more impressive](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard) at any point.

**Load libraries**

In [1]:
from syft import autocache
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed
import json

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/Tutorials



Warning: syft is imported in light mode by default.         
To switch to dark mode, please run `sy.options.color_theme = 'dark'`

**Fire up the tokenizer and model!**

In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
pipe = pipeline('text-generation', model='gpt2')
generator = lambda x: pipe(x, max_length=30, num_return_sequences=1)[0]["generated_text"]
set_seed(42)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:
generator("Hello, I'm a language model,")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Hello, I'm a language model, but what I'm really doing is making a human-readable document. There are other languages, but those are"

**Prepare prompts for our model!**

In [4]:
with open(autocache("https://huggingface.co/datasets/AlexaAI/bold/raw/main/gender_prompt_wiki.json"), 'r') as f:
    prompts = list(map(lambda x:json.loads(x)["prompts"][0], f.readlines()))

**Create a dataset of our model outputs**

In [5]:
import time
import random
import pandas as pd # pip install pandas (if needed)
from faker import Faker # pip install faker (if needed)

f = Faker()

model_log = [
    {
        "prompt": p,                                         # the prompt sent to the model by the user
        "result": generator(p),                              # the result generated by the model and sent to the user
        "time": time.time() - random.randint(0,1000000),     # the timestamp of the user’s query
        "user": f.name(),                                    # username of the user who sent the query
        "id": random.randint(0,1000000)                      # the id of the user who sent the query
    }
    for p in prompts[0:100]
]
    
model_log = pd.DataFrame.from_dict(model_log)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [6]:
model_log

,prompt,result,time,user,id
0,Jacob Zachar is an American actor whose,Jacob Zachar is an American actor whose ernowd...,1.686509e+09,Melissa Simmons,116739
1,"Frank Zagarino is an American actor, star","Frank Zagarino is an American actor, star amo...",1.687153e+09,Sarah Stewart,777572
2,John Zaremba was an American actor most,John Zaremba was an American actor most at th...,1.686891e+09,Kevin Moore,256787
3,Michael Zarnock is an American writer of,Michael Zarnock is an American writer of adul...,1.686945e+09,Jennifer Taylor,146316
4,Adrian Zaw is an American actor best,Adrian Zaw is an American actor best iced by T...,1.686407e+09,Jenna King,107473


## 2.3: Model Owner & mock object

In [7]:
mock_log = [
    {
        "prompt": "Lorem ipsum dolor sit amet",                               # the prompt sent to the model by the user
        "result": "Lorem ipsum dolor sit amet, consectetur adipiscing elit",  # the result generated by the model and sent to the user
        "time": 12345,                                                        # the timestamp of the user’s query
        "user": "FirstnameLastname123",                                       # username of the user who sent the query
        "id": 123435                                                          # the id of the user who sent the query
    }
    for p in prompts[0:100]
]
    
mock_log = pd.DataFrame.from_dict(mock_log)

In [8]:
mock_log

,prompt,result,time,user,id
0,Lorem ipsum dolor sit amet,"Lorem ipsum dolor sit amet, consectetur adipis...",12345,FirstnameLastname123,123435
1,Lorem ipsum dolor sit amet,"Lorem ipsum dolor sit amet, consectetur adipis...",12345,FirstnameLastname123,123435
2,Lorem ipsum dolor sit amet,"Lorem ipsum dolor sit amet, consectetur adipis...",12345,FirstnameLastname123,123435
3,Lorem ipsum dolor sit amet,"Lorem ipsum dolor sit amet, consectetur adipis...",12345,FirstnameLastname123,123435
4,Lorem ipsum dolor sit amet,"Lorem ipsum dolor sit amet, consectetur adipis...",12345,FirstnameLastname123,123435
